In [1]:

import pandas as pd

!pip install numerapi

from numerapi import NumerAPI

from pathlib import Path

#from sklearn.linear_model import Ridge
from sklearn.feature_selection import SelectKBest, f_regression

import pyarrow.parquet as pq

import os


TOURNAMENT_NAME= "first_large"
ERA_COL = "era"
TARGET_COL = "target_nomi_20"
DATA_TYPE_COL = "data_type"
EXAMPLE_PREDS_COL = "example_preds"
PREDICTION_NAME = "prediction"

id = "OML65REYFDPC5O7N22XCRP44BG2M74XH"
key = "YSTL455VERL7WZ4D7OQ6XEYEQN2MRCCICBMILNFP3DUZC4MSAS2WSH2MV7ED6WB3"

napi = NumerAPI(public_id=id,secret_key=key)


     |████████████████████████████████| 78 kB 3.7 MB/s             


In [2]:
from utils import save_stuff, load_stuff
# utilities


def score(df):
    return df[[TARGET_COL, PREDICTION_NAME]].corr(method="spearman")[TARGET_COL][PREDICTION_NAME]



In [3]:
current_round = napi.get_current_round(tournament=8)  # tournament 8 is the primary Numerai Tournament

print('Downloading dataset files...')

napi.download_dataset("numerai_tournament_data.parquet", f"tournament_data_{current_round}.parquet")
napi.download_dataset("numerai_validation_data.parquet", f"validation_data.parquet")

validation_data = pq.read_table('validation_data.parquet').to_pandas()
tournament_data = pq.read_table(f"tournament_data_{current_round}.parquet").to_pandas()

napi.download_dataset("example_validation_predictions.parquet", "example_validation_predictions.parquet")

model_name = f"model_target"
print(f"Checking for existing model '{model_name}'")
model = load_stuff(model_name)
selected_features = load_stuff('features')
if not model:
    print(f"model not found")

model_expected_features = selected_features

read_columns = model_expected_features + [ERA_COL, DATA_TYPE_COL, TARGET_COL]

#validation_data = pd.read_csv('validation_data.csv', usecols=read_columns)
validation_data[PREDICTION_NAME] = model.predict(validation_data[model_expected_features])

validation_preds = pq.read_table("example_validation_predictions.parquet").to_pandas()
validation_data[EXAMPLE_PREDS_COL] = validation_preds["prediction"]

validation_correlations = validation_data.groupby("era").apply(score)
print(f"On validation the correlation has mean {validation_correlations.mean()} and std {validation_correlations.std()}")


print('Predicting on tournament data')
tournament_data[PREDICTION_NAME] = model.predict(tournament_data[model_expected_features])

tournament_data[PREDICTION_NAME].to_csv("predictions.csv")





2022-06-19 00:03:35,464 INFO numerapi.utils: starting download
tournament_data_321.parquet: 582MB [00:20, 28.8MB/s]                            
2022-06-19 00:03:56,257 INFO numerapi.utils: target file already exists
2022-06-19 00:03:56,257 INFO numerapi.utils: download complete
2022-06-19 00:04:03,915 INFO numerapi.utils: target file already exists
2022-06-19 00:04:03,916 INFO numerapi.utils: download complete


Checking for existing model 'model_target'
On validation the correlation has mean 0.015585370936464444 and std 0.03345040207303254
Predicting on tournament data


In [22]:
predns = pd.DataFrame(tournament_data[PREDICTION_NAME])

print(list(predns.columns))

['prediction']


In [12]:
predns = pd.DataFrame(tournament_data[PREDICTION_NAME])

print(predns.head())
#print('uploading')

print('uploading')
submission_string = napi.upload_predictions(predns)
print(submission_string)






2022-06-19 00:36:58,698 INFO numerapi.base_api: uploading predictions...


id
n000101811a8a843    0.503110
n001e1318d5072ac    0.500117
n002a9c5ab785cbb    0.476395
n002ccf6d0e8c5ad    0.510106
n0051ab821295c29    0.504429
Name: prediction, dtype: float32
uploading


2022-06-19 00:37:02,525 ERROR numerapi.base_api: Invalid submission headers. Headers must be id and prediction.


ValueError: Invalid submission headers. Headers must be id and prediction.